In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingRegressor
from tensorflow import keras
import os
from PIL import Image

import matplotlib.pyplot as plt
import requests
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow import keras
from keras import metrics


from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


In [ ]:
from sklearn.ensemble import BaggingRegressor

def build_bagging_model(base_estimator, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, random_state=0):
    """
    Build a bagging regressor model
    :param base_estimator: The base model to ensemble
    :param n_estimators: The number of base estimators in the ensemble
    :param max_samples: The fraction of samples to draw from X to train each base estimator
    :param max_features: The fraction of features to draw from X to train each base estimator
    :param bootstrap: Whether samples are drawn with replacement
    :param bootstrap_features: Whether features are drawn with replacement
    :param random_state: Controls the randomness of the estimator
    :return: A bagging regressor model
    """
    model = BaggingRegressor(
        base_estimator=base_estimator,
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        bootstrap=bootstrap,
        bootstrap_features=bootstrap_features,
        random_state=random_state
    )
    return model

# Example usage with a Decision Tree Regressor as the base model
dt_regressor = DecisionTreeRegressor(max_depth=10, random_state=0)
bagging_model = build_bagging_model(dt_regressor)


In [ ]:
# Fit the bagging regressor
bagging_model.fit(x_train_flat, y_train_flat)


In [ ]:
# Predict with the bagging regressor
y_pred_ts_bagging = bagging_model.predict(x_test_flat)


In [ ]:
# Evaluate the bagging regressor
mse_bagging = calculate_mse(y_pred_ts_bagging, y_test_flat)
rmse_bagging = np.sqrt(mse_bagging)
mae_bagging = calculate_mae(y_pred_ts_bagging, y_test_flat)
r2_bagging = calculate_r2(y_pred_ts_bagging, y_test_flat)

print("Bagging MSE: ", mse_bagging)
print("Bagging RMSE: ", rmse_bagging)
print("Bagging MAE: ", mae_bagging)
print("Bagging R2: ", r2_bagging)
